# Peak finding

## Simulated image of galaxies and stars

In the first part of this notebook we consider an image that includes 100 sources, all with Gaussian profiles, some of which are fairly elongated. We used this image in the previous section about removing the background prior to source detection.

As usual, we begin with some imports.

In [ ]:
import warnings

from astropy.convolution import convolve
from astropy.stats import sigma_clipped_stats, gaussian_sigma_to_fwhm
from astropy.table import QTable
from astropy.visualization import simple_norm, SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

import matplotlib.pyplot as plt
import numpy as np

from photutils.aperture import CircularAperture, EllipticalAperture
from photutils.centroids import centroid_2dg
from photutils.datasets import make_100gaussians_image, make_gaussian_sources_image
from photutils.detection import find_peaks, DAOStarFinder
from photutils.segmentation import detect_sources, make_2dgaussian_kernel, SourceCatalog
from photutils.utils import make_random_cmap

plt.style.use('../photutils_notebook_style.mplstyle')

To begin, let's create the image and subtract the background. Recall that this image contains a mix of star-like object and more extended sources.

In [ ]:
data = make_100gaussians_image()
mean, med, std = sigma_clipped_stats(data, sigma=3.0, maxiters=5)
data_subtracted = data - med

In [ ]:
plt.style.use('../photutils_notebook_style.mplstyle')

## Source Detection with peak finding

For more general source detection cases that do not require comparison with models, `photutils` offers the [`find_peaks`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks) function. 

This function simply finds sources by identifying local maxima above a given threshold and separated by a given distance, rather than trying to fit data to a given model. Unlike the previous detection algorithms, [`find_peaks`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks) does not necessarily calculate objects' centroids. Unless the `centroid_func` argument is passed a function like `photutils.centroids.centroid_2dg` that can handle source position centroiding, `find_peaks` will return just the integer value of the peak pixel for each source.

This algorithm is particularly useful for identifying non-stellar sources or heavily distorted sources in image data.

The centroid-finding, which we do here only so that we can compare positions found by this method to positions found by the star-finding methods, can generate many warnings when an attempt is made to fit a 2D Gaussian to the sources. The warnings are suppressed below to avoid cluttering the output.

Let's see how it does:

In [ ]:
med, std

In [ ]:
# Define threshold and minimum object size
threshold = 3. * std
npixels = 10
fwhm = 3
kernel = make_2dgaussian_kernel(fwhm, size=5)
convolved_data = convolve(data_subtracted, kernel)
# Create a segmentation image
segm = detect_sources(convolved_data, threshold, npixels)

print('Found {} sources'.format(segm.max_label))

In [ ]:
with warnings.catch_warnings(action="ignore"):
    sources_findpeaks = find_peaks(data_subtracted,
                                   threshold=5. * std, box_size=29,
                                   centroid_func=centroid_2dg)
print(sources_findpeaks)

In [ ]:
# Set up the figure with subplots
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
plt.tight_layout()

# Plot the data
# Set up the normalization and colormap
norm_image = ImageNormalize(stretch=SqrtStretch())

# Plot the original data
fitsplot = ax1.imshow(data_subtracted,
                      norm=norm_image, cmap='Greys')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Original Data')

# Plot the segmentation image
rand_cmap = make_random_cmap(seed=12345)
rand_cmap.set_under('black')
segplot = ax2.imshow(segm, vmin=1, cmap=rand_cmap)
ax2.set_xlabel('X (pixels)')
ax2.set_title('Segmentation Map')

# Define the colorbar
cbar_ax = fig.add_axes([1, 0.09, 0.03, 0.87])
cbar = plt.colorbar(segplot, cbar_ax)
cbar.set_label('Object Label', rotation=270, labelpad=30)

## Comparing Detection Methods

Let's compare how peak finding compares to using [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) on this image.


In [ ]:
daofind = DAOStarFinder(fwhm=fwhm, threshold=threshold) 
sources_dao = daofind(data_subtracted)

In [ ]:
print(f'''DAOStarFinder: {len(sources_dao)} sources
find_peaks: {len(sources_findpeaks)} sources''') 

In [ ]:
catalog = SourceCatalog(data_subtracted, segm)
table = catalog.to_table()

# Define the approximate isophotal extent
r = 4.  # pixels

# Create the apertures
apertures = []
for obj in catalog:
    position = (obj.xcentroid, obj.ycentroid)
    a = obj.semimajor_sigma.value * r
    b = obj.semiminor_sigma.value * r
    theta = obj.orientation
    apertures.append(EllipticalAperture(position, a, b, theta=theta))

And just for fun, let's plot these matching sources. (The colors chosen to represent different sets are from [Paul Tol's guide for accessible color schemes](https://personal.sron.nl/~pault/).)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(data, norm=norm_image, cmap='Greys', origin='lower')

marker_size = 60
ax1.scatter(sources_findpeaks['x_peak'], sources_findpeaks['y_peak'], s=marker_size, marker='s',
            lw=1, alpha=1, facecolor='None', edgecolor='r', label='Found by find_peaks')
ax1.scatter(sources_dao['xcentroid'], sources_dao['ycentroid'], s=2 * marker_size, marker='D',
            lw=1, alpha=1, facecolor='None', edgecolor='#0077BB', label='Found by DAOfind')

# Plot the apertures
apertures[0].plot(color='red', lw=1, alpha=1, axes=ax1, label="Image segmentation")
for aperture in apertures[1:]:
    aperture.plot(color='cyan', lw=1, alpha=1, axes=ax1)
    
# Add legend
ax1.legend(ncol=2, loc='lower center', bbox_to_anchor=(0.5, -0.35))


ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Sources Found by Different Methods');

## Summary

Image segmentation identified more of the sources in this test image than either [`find_peaks`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks) or [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder). The latter wasn't expected to work well on this image because there is a large variety of source shapes and sizes.